In [224]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
import pickle
import re

In [225]:
vote_file = pd.read_csv('brexit_full_updated3.csv')
vote_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN


In [226]:
politicians_party = {}
politicians = vote_file['full name'].unique()
for index, row in vote_file.iterrows():
    politicians_party[row['full name']] = row['party']

In [227]:
def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return t
        
    return topic

In [446]:
df1 = pd.read_csv('2015_commons.csv')
df2 = pd.read_csv('2016_commons.csv')
df = pd.concat([df1, df2])
df = df.drop(['Government'], axis=1)
df = df.loc[(df['Party'] == 'Conservative') | (df['Party'] == 'Labour')]
df = df.loc[df['Name'].isin(politicians)]
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] > '2015-5-1') & (df['Date'] <= '2016-6-30')
df = df.loc[mask]
df['party2'] = df['Name'].apply(lambda x: politicians_party[x])
df['Topic'] = df['Topic'].apply(lambda x: preprocess(x))
# df = df.loc[df['Topic'] == 'Parliament, government and politics']
# df = df.loc[df['Topic'] == 'European Union']
# df = df.loc[df['Topic'] == 'Crime, civil law, justice and rights']
# df = df.loc[df['Topic'] == 'Health services and medicine']
df = df.loc[df['Topic'] == 'International affairs']
cols = df.columns
cols = ['date', 'speaker', 'name', 'Party', 'Topic', 'transcript', 'party2']
df.columns = cols
df.head()

,date,speaker,name,Party,Topic,transcript,party2
19003,2015-06-01,Kevin Brennan (Cardiff West) (Lab),Kevin Brennan,Labour,International affairs,The Secretary of State mentioned in his preamb...,0.0
19010,2015-06-01,Rehman Chishti (Gillingham and Rainham) (Con),Rehman Chishti,Conservative,International affairs,My right hon. Friend says that we should not l...,1.0
19012,2015-06-01,"Ms Gisela Stuart (Birmingham, Edgbaston) (Lab)",Gisela Stuart,Labour,International affairs,Is this an opportunity for the Foreign Secreta...,1.0
19015,2015-06-01,Mr Peter Bone (Wellingborough) (Con),Peter Bone,Conservative,International affairs,I am grateful to the Foreign Secretary for giv...,1.0
19022,2015-06-01,Mike Gapes (Ilford South) (Lab/Co-op),Mike Gapes,Labour,International affairs,Has the Foreign Secretary seen the recent repo...,0.0


In [447]:
df['party'] = df['party2'].apply(lambda x: 'leave' if x == 1 else 'remain')

In [448]:
rows = []

for index, row in df.iterrows():
    if row['party2'] == 1:
        party = 'leave'
        rows.append([row['name'], party])
    elif row['party2'] == 0:
        party = 'remain'
        rows.append([row['name'], party])
    
    
politicians_chosen = pd.DataFrame(rows, columns=['full name', 'party'])
politicians_chosen = politicians_chosen.drop_duplicates().reset_index(drop=True)
politicians_chosen['party'] = politicians_chosen['party'].apply(lambda x: 0 if x == 'leave' else 1)
politicians_chosen


,full name,party
0,Kevin Brennan,1
1,Rehman Chishti,0
2,Gisela Stuart,0
3,Peter Bone,0
4,Mike Gapes,1
...,...,...
155,Nick Herbert,1
156,Nigel Adams,0
157,Kelvin Hopkins,0
158,Matt Warman,1


In [449]:
train_df = df.loc[(df.party == 'leave') | (df.party == 'remain')]

In [450]:
train_df

,date,speaker,name,Party,Topic,transcript,party2,party
19003,2015-06-01,Kevin Brennan (Cardiff West) (Lab),Kevin Brennan,Labour,International affairs,The Secretary of State mentioned in his preamb...,0.0,remain
19010,2015-06-01,Rehman Chishti (Gillingham and Rainham) (Con),Rehman Chishti,Conservative,International affairs,My right hon. Friend says that we should not l...,1.0,leave
19012,2015-06-01,"Ms Gisela Stuart (Birmingham, Edgbaston) (Lab)",Gisela Stuart,Labour,International affairs,Is this an opportunity for the Foreign Secreta...,1.0,leave
19015,2015-06-01,Mr Peter Bone (Wellingborough) (Con),Peter Bone,Conservative,International affairs,I am grateful to the Foreign Secretary for giv...,1.0,leave
19022,2015-06-01,Mike Gapes (Ilford South) (Lab/Co-op),Mike Gapes,Labour,International affairs,Has the Foreign Secretary seen the recent repo...,0.0,remain
...,...,...,...,...,...,...,...,...
35408,2016-06-29,Andy Slaughter (Hammersmith) (Lab),Andy Slaughter,Labour,International affairs,T3. Given what the Overseas Development Inst...,0.0,remain
35415,2016-06-29,Mr Alan Mak (Havant) (Con),Alan Mak,Conservative,International affairs,T6. Encouraging business growth in developin...,0.0,remain
35417,2016-06-29,Wes Streeting (Ilford North) (Lab),Wes Streeting,Labour,International affairs,Given the support that the Government provide ...,0.0,remain
35419,2016-06-29,Mark Spencer (Sherwood) (Con),Mark Spencer,Conservative,International affairs,T7. Will the Secretary of State update the H...,0.0,remain


In [451]:
def get_bigrams(text):
    # To remove stop words
    stopW = stopwords.words('english')

    # To stem
    ps = PorterStemmer()

    # Tokenize

    clean_transcript = text.lower()
    clean_transcript = word_tokenize(clean_transcript)
    # Remove digits
    clean_transcript = [i for i in clean_transcript if not re.match(r'\d+', i)]
    # Remove Stopwords and single characters
    clean_transcript = [i for i in clean_transcript if i not in stopW and len(i) > 1]
    # Stemming
    clean_transcript = [ps.stem(word) for word in clean_transcript]

    bigrams = []
    
    _len = len(clean_transcript)
    
    for i in range(_len-1):
        bigrams.append(clean_transcript[i] + '.' + clean_transcript[i+1])
        
    return bigrams

In [452]:
all_bigrams = []

n = len(train_df)

for index in tqdm(range(n)):
    row = train_df.iloc[index]
    all_bigrams.extend(get_bigrams(row['transcript']))
    all_bigrams = list(set(all_bigrams))
    
len(all_bigrams)

100%|██████████| 1599/1599 [00:08<00:00, 192.97it/s]


72665

In [453]:
with open('all_bigrams.pkl', 'wb') as f:
    pickle.dump(all_bigrams, f)

In [454]:
with open('all_bigrams.pkl', 'rb') as f:
    all_bigrams = pickle.load(f)

In [455]:
politicians_chosen

,full name,party
0,Kevin Brennan,1
1,Rehman Chishti,0
2,Gisela Stuart,0
3,Peter Bone,0
4,Mike Gapes,1
...,...,...
155,Nick Herbert,1
156,Nigel Adams,0
157,Kelvin Hopkins,0
158,Matt Warman,1


In [456]:
politicians = politicians_chosen['full name'].values

In [457]:
count_bigrams_by_politicians = {bigram : {politician: 0 for politician in politicians} for bigram in all_bigrams}

In [458]:
n = len(train_df)
for index in tqdm(range(n)):
    row = train_df.iloc[index]
    bigrams = get_bigrams(row['transcript'])
    politician = row['name']
    if politician not in politicians:
        continue
    for bigram in bigrams:
        count_bigrams_by_politicians[bigram][politician] += 1


100%|██████████| 1599/1599 [00:05<00:00, 268.36it/s]


In [459]:
with open('count_bigrams_by_politicians.pkl', 'wb') as f:
    pickle.dump(count_bigrams_by_politicians, f)

In [460]:
with open('count_bigrams_by_politicians.pkl', 'rb') as f:
    count_bigrams_by_politicians = pickle.load(f)


In [461]:
bigrams = list(count_bigrams_by_politicians.keys())
for bigram in bigrams:
    summ = sum(count_bigrams_by_politicians[bigram].values())
    if summ < 5:
        count_bigrams_by_politicians.pop(bigram, None)
        print('removed {}'.format(bigram))

removed number.migrant
removed somaliland.import
removed defeat.isil/daesh
removed feel.argu
removed trump.figur
removed support.educ
removed next.month
removed fellow.citizen
removed problem.around
removed stori.young
removed current.focus
removed approach.drastic
removed deport.serv
removed displac.futur
removed manifesto.absolut
removed histori.instabl
removed oppress.day
removed travel.pleas
removed market.therefor
removed watch.somerset
removed salafist.group
removed reconstruct.plan
removed offic.previou
removed look.steal
removed timescal.given
removed work.nobel
removed problem.push
removed told.prime
removed indian.pakistani
removed jamaica.jamaican
removed say.need
removed call.urgent
removed religion.civilian
removed inde.prime
removed debat.middl
removed life-sav.emerg
removed compani.essenc
removed claus.bill
removed requir.secretari
removed educ.brief
removed substant.motion
removed east.even
removed peac.impact
removed continent.word
removed share.argument
removed occur.

removed nobodi.would
removed iraqi.armi
removed saudi.ensur
removed circumst.discussing—a
removed one.oldest
removed mainstay.daesh
removed govern.ivori
removed great.signific
removed respons.area
removed us.tomorrow
removed hous.felt
removed bonus.howev
removed approv.refer
removed doctor.child
removed train.incorrectli
removed authoris.forc
removed express.major
removed sourc.econom
removed evil.minist
removed daesh.action
removed wrote.recent
removed bill.prescrib
removed discuss.school
removed decis.paus
removed citi.council
removed court.sent
removed harass.local
removed offenc.clearli
removed pass.test
removed mr.wilkinson
removed g7.agenda
removed known.friend
removed offic.rightli
removed free.£500
removed author.world
removed show.two
removed serv.committe
removed intent.quibbl
removed misconduct.proceed
removed solut.long-term
removed state.thought
removed point.uniqu
removed sinc.led
removed help.bail
removed school.cpr
removed rais.manag
removed protect.place
removed virtua

removed fact.hard-won
removed ladi.hon
removed small.small
removed man.yet
removed get.import
removed caus.part
removed arriv.britain
removed move.within
removed time.acknowledg
removed debat.must
removed thing.year
removed complet.starvat
removed offic.lost
removed iran.mani
removed individu.age
removed vigour.shock
removed bring.parti
removed hous.tri
removed think.sort
removed english.court
removed seen.progress
removed argument.afraid
removed voluntari.organis
removed turkey.human
removed begin.believ
removed need.diplomat
removed underscor.liberti
removed job.singl
removed place.home
removed stop.tide
removed isil-first—in.word
removed speak.colleagu
removed achiev.noth
removed pillar.unit
removed rais.money
removed area.unless
removed care.judgment
removed adopt.polici
removed whatev.natur
removed achiev.polit
removed hand.children
removed treati.broken
removed island.set
removed support.europol
removed take.fingerprint
removed defenc.cours
removed ahmed-sheikh.hon
removed mean.b

removed non-extremist.opposit
removed crimin.uk
removed benefit.hope
removed suspect.opposit
removed clear.six
removed explain.think
removed made.reason
removed tradit.industri
removed key.test
removed briton.unabl
removed west.assur
removed place.ministri
removed fragil.conflict
removed protect.syrian
removed world.talk
removed last.octob
removed commend.fact
removed spoken.eloqu
removed histor.role
removed happen.nuclear
removed action.across
removed one.criteria
removed appropri.concentr
removed side.hon
removed whether.airstrik
removed import.judg
removed differ.employ
removed appear.moment
removed southwel.train
removed lee.bo
removed proper.first
removed secondli.reduc
removed mention.elabor
removed number.returne
removed mani.younger
removed nation.object
removed question.much
removed custom.senior
removed major.western
removed crisi.constitu
removed syria.cut
removed scath.report
removed berwick.first
removed sinc.open
removed busi.govern
removed chang.rwanda
removed primari.fa

removed found.littl
removed liter.two
removed believe—that.tonight
removed certain.age
removed prison.look
removed gentleman.knowledg
removed yet.comment
removed care.commun
removed uniqu.good
removed challeng.uk
removed read.debat
removed place.behalf
removed so-cal.deal
removed balfour.declar
removed instead.govern
removed exist.larg
removed more..give
removed like.barrier
removed accept.public
removed non-compli.countri
removed countri.treat
removed kingdom.german
removed mosqu.action
removed far-right.extremism—to
removed occupi.countri
removed south.ruislip—aft
removed elit.complet
removed iraqi.kurd
removed parent.teacher
removed address.mass
removed citizen.proportion
removed addit.approach
removed fli.wing
removed secretari.use
removed along.head
removed love.give
removed popul.plummet
removed give.commit
removed extremist.middl
removed mere.sleep
removed scheme.prime
removed nobl.lord
removed erdogan.crack
removed europhob.doorstep
removed mission.found
removed commit.spend
re

...

removed aris.question
removed gaza.ensur
removed must.accept
removed action.area
removed sicken.scene
removed kara-murza.close
removed rather.mere
removed believ.it—or
removed nation.open
removed contain.defeat
removed alli.suffer
removed case.might
removed locat.constitu
removed thank.excel
removed forc.african
removed better.take
removed matter.renew
removed umbrella.legitimaci
removed way.departur
removed first.pay
removed attent.want
removed complet.sincer
removed parti.among
removed one.someon
removed chang.biometr
removed terrorist.silent
removed place.greater
removed speech.prais
removed ask.parliamentari
removed word.deterr
removed societi.trade
removed practic.courag
removed commonwealth—th.ex-british
removed possibl.ensur
removed go.iraq
removed confer.signific
removed member.wait
removed commiss.group
removed competitor.slow
removed hard-work.jamaican
removed match.eloqu
removed end.term
removed peopl.recognis
removed russia.nationalist
removed jeremi.brown
removed impli.ref

removed damag.least
removed agricultur.polici
removed len.mccluskey
removed iran.nato
removed take.mediterranean
removed someth.probabl
removed day.britain
removed women.especi
removed invest.month
removed made.earlier
removed earli.lifetim
removed war..precis
removed within.two
removed yemen.among
removed lack.documentari
removed discov.least
removed vision.realiti
removed secur.safeti
removed said.bloomberg
removed live.ground
removed shown.use
removed peac.decenc
removed innov.come
removed foil.far
removed citizen.well
removed eu.competit
removed allow.stay
removed way.truli
removed encourag.pursu
removed fli.iraq
removed minist.prais
removed plan.real
removed support.agre
removed minist.wait
removed favour.establish
removed report.pshe
removed street.question
removed passion.grimsbi
removed obstruct.tit-for-tat
removed robust.regul
removed interpret.modern
removed like.phoney
removed high-qual.first
removed die.mani
removed done.prevent
removed way.peac
removed provis.european
remo

removed huge.invest
removed receiv.dignifi
removed show.foreign
removed draw.hous
removed ebola.crisis—in
removed fail.tackl
removed alreadi.function
removed opposit.troop
removed russia.stop
removed might.desir
removed regiment.take
removed tri.work
removed life.get
removed week.recent
removed tomorrow.solv
removed remov.foster
removed bear.turkey
removed one.sent
removed children.arriv
removed two-term.presidenti
removed dramat.effect
removed death.palestinian
removed simpl.point
removed mani.young
removed includ.venu
removed address.spend
removed hous.—i
removed problem.face
removed alway.oppos
removed mission.moscow
removed vulner.children
removed councillor.plymouth
removed wish.learn
removed public.secur
removed improv.relat
removed throughout.lives—they
removed implic.wide
removed sensibl.simpl
removed prison.full
removed shore.got
removed breakdown.contin
removed enough.acknowledg
removed detail.explan
removed key.question
removed dark.heard
removed sport.music
removed shore.or

removed million.interest
removed met.founder
removed casualti.strike
removed prosper.fulfil
removed penuri.miseri
removed justic.irrevers
removed billion.felt
removed post.fulfil
removed move.planet
removed expertis.use
removed project.stop
removed offend.britain
removed huge.degre
removed want.today
removed committe.result
removed region.work
removed tenuou.human
removed safeguard.peopl
removed level.democraci
removed newark.return
removed within.home
removed express.deep
removed carri.review
removed secretari.eloqu
removed develop.made
removed moroccan.woman
removed union.deem
removed wife.yet
removed detract.proxi
removed valu.differ
removed reason.withdrew
removed everyon.chanc
removed solut.defenc
removed uk.big
removed let.quot
removed interestingli.report
removed tremend.asset
removed us.much
removed home.without
removed girlfriend.young
removed come.counti
removed cloak.reinforc
removed use.circumv
removed valu.citizen
removed taught.system
removed present.scale
removed growth.

removed parti.link
removed crucibl.christian
removed compulsori.first
removed market.british
removed visa.point
removed focus.effort
removed australian.govern
removed like.foreign
removed among.thing
removed back.detail
removed order.british
removed exactli.provid
removed govern.probabl
removed use.recent
removed union.abl
removed real.present
removed anoth.problem
removed influenc.nation
removed arm.race
removed third.mean
removed essenti.life
removed hous.way
removed reason.basic
removed lithuanian.come
removed khartoum.process
removed intelligence—th.five
removed citizen.convict
removed attack.two
removed per.se
removed wrong.delight
removed true.northumbrian
removed agenda.decemb
removed africa.would
removed right.privaci
removed bill.put
removed possibl.co-ordin
removed moment.one
removed everi.occas
removed help.happen
removed principl.equiti
removed amount.relief
removed ambassador.iran
removed anoth.issu
removed view.someth
removed oppos.entir
removed million.crimin
removed imp

In [462]:
print(len(politicians))
print(len(cols))

160
7


In [463]:
['bigrams'].extend(politicians)

In [464]:
cols = ['bigrams']
cols.extend(politicians)
rows = []
for key in count_bigrams_by_politicians.keys():
    row = [key]
    for politician in politicians:
        row.append(count_bigrams_by_politicians[key][politician])
    rows.append(row)
res_df = pd.DataFrame(rows, columns=cols)
res_df

,bigrams,Kevin Brennan,Rehman Chishti,Gisela Stuart,Peter Bone,Mike Gapes,Jim Cunningham,Henry Smith,Keith Vaz,David Burrowes,...,Angela Rayner,Rupa Huq,Kevin Hollinrake,Kevan Jones,Oliver Heald,Nick Herbert,Nigel Adams,Kelvin Hopkins,Matt Warman,Andy Burnham
0,give.hous,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,first.duti,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,train.voluntari,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ministri.justic,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,govern.motion,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,take.part,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1313,eea.nation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1314,foreign.prison,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1315,might.get,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [465]:
politicians_chosen

,full name,party
0,Kevin Brennan,1
1,Rehman Chishti,0
2,Gisela Stuart,0
3,Peter Bone,0
4,Mike Gapes,1
...,...,...
155,Nick Herbert,1
156,Nigel Adams,0
157,Kelvin Hopkins,0
158,Matt Warman,1


In [466]:
def calculateChiSq(politicians_chosen, political_Ngrams_20, all_=False, equal_conservative_labour_Ngrams=True, topic=""):

    # we give a file with full list of politicians but choose those that have party info below either way
    lc_names = list(politicians_chosen.loc[politicians_chosen["party"] == 0]["full name"])
    rl_names = list(politicians_chosen.loc[politicians_chosen["party"] == 1]["full name"])

    # we throw away politicians without party info so we delete bigrams that are spoken
    # by none of the politicians that are left to avoid division by zero
    politicians_with_party_info = list(politicians_chosen.dropna(subset=["party"])["full name"].values)
    political_Ngrams_20 = political_Ngrams_20.loc[
                          (political_Ngrams_20[politicians_with_party_info].sum(axis=1) > 0), :]

    C_bc = political_Ngrams_20[lc_names]
    C_bl = political_Ngrams_20[rl_names]

    C_bc = C_bc.sum(axis=1)
    C_bl = C_bl.sum(axis=1)

    C_c = C_bc.sum(axis=0)
    C_l = C_bl.sum(axis=0)

    is_conservative = C_bc.values >= C_bl.values

    print("Total leave bigrams: " + str(C_c))
    print("Total remain bigrams: " + str(C_l))

    C_nbc = C_c - C_bc
    C_nbl = C_l - C_bl

    den = ((C_bc + C_bl) * (C_bc + C_nbc) * (C_bl + C_nbl) * (C_nbc + C_nbl))
    coeffs = (C_bc * C_nbl - C_bl * C_nbc) ** 2 / den

    Ngrams = list(political_Ngrams_20['bigrams'])
    coeffs = pd.DataFrame(
        {'bigrams': Ngrams, "chi sq": list(coeffs), "leave": list(is_conservative)})

    print("Sorting values.", flush=True)
    if equal_conservative_labour_Ngrams:
        coeffs_conservative = coeffs.loc[coeffs["leave"] == True, :]
        coeffs_labour = coeffs.loc[coeffs["leave"] == False, :]

        coeffs_conservative.sort_values(by=['chi sq', 'bigrams'], axis=0, ascending=False, inplace=True,
                                        kind='quicksort', na_position='last')
        coeffs_labour.sort_values(by=['chi sq', 'bigrams'], axis=0, ascending=False, inplace=True, kind='quicksort',
                                  na_position='last')

        if topic == "":
            take_no_Ngrams = 600
        else:
            take_no_Ngrams = 300

        coeffs_conservative = coeffs_conservative.iloc[:take_no_Ngrams, :]
        coeffs_labour = coeffs_labour.iloc[:take_no_Ngrams, :]
        coeffs_conservative_Ngrams = coeffs_conservative['bigrams'].values
        coeffs_labour_Ngrams = coeffs_labour['bigrams'].values
        total_Ngrams = list(coeffs_conservative_Ngrams) + list(coeffs_labour_Ngrams)

        coeffs = coeffs.loc[coeffs['bigrams'].isin(total_Ngrams), :]
        coeffs.sort_values(by=['chi sq', 'bigrams'], axis=0, ascending=False, inplace=True, kind='quicksort',
                           na_position='last')

    else:
        coeffs.sort_values(by=['chi sq', 'bigrams'], axis=0, ascending=False, inplace=True, kind='quicksort',
                           na_position='last')
        coeffs.to_csv('./all_bigrams.csv')
        if not all_:
            coeffs = coeffs.iloc[:1000, :]

    return coeffs.reset_index(drop=True)

In [467]:
coeffs = calculateChiSq(politicians_chosen,  res_df)

Total leave bigrams: 8026
Total remain bigrams: 6797
Sorting values.


/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [468]:
coeffs

,bigrams,chi sq,leave
0,foreign.nation,0.007248,True
1,hon.friend,0.005320,True
2,nation.offend,0.004779,True
3,secretari.state,0.002795,False
4,prime.minist,0.002162,False
...,...,...,...
1145,either.side,0.000005,True
1146,diplomat.effort,0.000005,True
1147,day.later,0.000005,True
1148,daesh.would,0.000005,True


In [469]:
coeffs.to_csv('leave_and_remain_coeffs.csv', index=False)

In [470]:
top_bigrams = coeffs['bigrams'].values

In [471]:
res_df_1000 = res_df.loc[res_df['bigrams'].isin(top_bigrams)].reset_index(drop=True)
res_df_1000

,bigrams,Kevin Brennan,Rehman Chishti,Gisela Stuart,Peter Bone,Mike Gapes,Jim Cunningham,Henry Smith,Keith Vaz,David Burrowes,...,Angela Rayner,Rupa Huq,Kevin Hollinrake,Kevan Jones,Oliver Heald,Nick Herbert,Nigel Adams,Kelvin Hopkins,Matt Warman,Andy Burnham
0,give.hous,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,first.duti,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,train.voluntari,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ministri.justic,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,govern.motion,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,tax.treati,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1146,may.impos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1147,eea.nation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1148,foreign.prison,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [472]:
res_df.drop(['bigrams'], axis=1).sum(axis=0)

Kevin Brennan      27
Rehman Chishti    140
Gisela Stuart     154
Peter Bone         65
Mike Gapes        158
                 ... 
Nick Herbert       14
Nigel Adams         8
Kelvin Hopkins      3
Matt Warman        11
Andy Burnham      202
Length: 160, dtype: int64

In [473]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV

def standardize(political_Ngrams_1000):
    Ngrams = political_Ngrams_1000['bigrams']
    col_names = political_Ngrams_1000.columns
    
    political_Ngrams_1000_politicians_only = political_Ngrams_1000.drop(['bigrams'], axis=1).fillna(0)
    
    political_Ngrams_1000_std = political_Ngrams_1000_politicians_only / political_Ngrams_1000_politicians_only.sum(axis=0)
    std = np.std(political_Ngrams_1000_std, axis=1)
    mean = np.mean(political_Ngrams_1000_std, axis=1)
    political_Ngrams_1000_std = (political_Ngrams_1000_std.T - np.mean(political_Ngrams_1000_std, axis=1)).T
    
    political_Ngrams_1000_std = (political_Ngrams_1000_std.T / std).T
    political_Ngrams_1000_std = pd.DataFrame(data=np.column_stack(
    (Ngrams, political_Ngrams_1000_std)), columns=col_names)
    
    mean_std = pd.DataFrame(data=np.column_stack(
    (Ngrams, mean, std)), columns=['bigrams', 'mean', 'std'])
    
    return political_Ngrams_1000_std, mean_std

def calculateBetas(political_Ngrams_1000, political_Ngrams_1000_std, save_alphas_betas=False):

    regressor = LinearRegression(n_jobs=1)
    regressor_no_intercept = LinearRegression(fit_intercept=False, n_jobs=1)

    def regress(X, political_Ngrams_1000, political_Ngrams_1000_std, name=""):

        # remove bigrams with zero row sum for politicians and remove sources with column sum zero that will
        # come to existance after bigrams removal
        political_Ngrams_1000_nonzero_bigrams = list(political_Ngrams_1000.loc[ \
                                                                 (political_Ngrams_1000.drop(
                                                                     'bigrams', axis=1).sum(axis=1) > 0),
                                                                 :]['bigrams'].values)

        political_Ngrams_1000 = political_Ngrams_1000.loc[political_Ngrams_1000['bigrams'].isin(
            political_Ngrams_1000_nonzero_bigrams), :]

        political_Ngrams_1000_std = political_Ngrams_1000_std.loc[political_Ngrams_1000_std[ \
                                                    'bigrams'].isin(
                                                    political_Ngrams_1000_nonzero_bigrams), :]

        political_Ngrams_1000_std = political_Ngrams_1000_std.loc[
                                    political_Ngrams_1000_std['bigrams'].isin(
                                        political_Ngrams_1000_nonzero_bigrams), :]

        # calculate Linear Regression
        Ngrams = list(political_Ngrams_1000_std['bigrams'].values)
        political_Ngrams_1000_std.drop('bigrams', axis=1, inplace=True)

        betas = []
        alphas = []
        for i in range(political_Ngrams_1000_std.shape[0]):
            y = np.array(political_Ngrams_1000_std.iloc[i, :]).ravel()
            print(len(X))
            print(len(y))
            regressor.fit(X, y)
            betas.append(regressor.coef_[0])
            alphas.append(regressor.intercept_)

        alphas = np.array(alphas)
        betas = np.array(betas)

        # Save partial information
        if (save_alphas_betas == True):
            final_df = political_Ngrams_1000_std
            final_df.to_csv("./final_dataframe_" + name + "_" + "std_after" + ".csv", index=False)

            alphas_betas_df = pd.DataFrame({'bigrams': Ngrams, "alphas": list(alphas), \
                                            "betas": list(betas),
                                            })
            alphas_betas_df.to_csv("./alphas_betas_" + name + "_" + "std_after" + ".csv", index=False)

        return alphas, betas
    
    X = np.array(politicians_chosen.dropna(subset=["party"])["party"]).reshape(-1, 1)
    politician_final_names = list(politicians_chosen.dropna(subset=["party"])["full name"].values)
    alphas_a, betas_a = regress(X, political_Ngrams_1000, political_Ngrams_1000_std, "party")

    regression_parameters = {"alphas_party": alphas_a, "betas_party": betas_a}

    return regression_parameters

def chooseNames(political_Ngrams_1000_std):
    
    global politicians_chosen
    politicians = pd.read_csv("./brexit_full_updated3.csv")

    politicians = politicians[["full name", "party", "conShare", "conShare2"]]
    politicians = politicians.replace("NA", np.nan)

    # drop politicians that do not have information about "party" and "conShare" and "conShare2" from info dataframe
    politicians = politicians.dropna(subset=["party"], how="all")

    common_names = political_Ngrams_1000_std.columns[1:]

    politicians_chosen = politicians_chosen.loc[politicians_chosen["full name"].isin(common_names)]
    
    print(len(common_names))
    return politicians_chosen

In [474]:
len(cols)

161

In [475]:
res_df_1000 = res_df_1000.loc[:, [True] + list(
            (res_df_1000.drop('bigrams', axis=1).sum(axis=0) > 0).values)]

political_Ngrams_1000_std, std_params = standardize(res_df_1000)
politicians_chosen = chooseNames(political_Ngrams_1000_std).drop_duplicates(subset=['full name'])
cols = political_Ngrams_1000_std.columns[1:]

names = politicians_chosen['full name'].values
cols = [col for col in cols if col in names]
columns = ['bigrams']
columns.extend(cols)
# politicians_chosen2 = politicians_chosen2.loc[politicians_chosen2['full name'].isin(cols)]
political_Ngrams_1000_std = political_Ngrams_1000_std[columns]
res_df_1000 = res_df_1000[columns]
# display(politicians_chosen)
print("Calculating Betas.")

regression_parameters = calculateBetas(res_df_1000, political_Ngrams_1000_std, save_alphas_betas=True)

159
Calculating Betas.
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
1

159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159
159


In [476]:
regression_parameters

{'alphas_party': array([-0.13886545, -0.06633915,  0.00731502, ...,  0.03276063,
         0.02501223,  0.22036717]),
 'betas_party': array([ 0.20829817,  0.09950872, -0.01097253, ..., -0.04914095,
        -0.03751835, -0.33055076])}

In [477]:
alphas = regression_parameters['alphas_party']
betas = regression_parameters['betas_party']
top_bigrams = res_df_1000['bigrams'].values

In [478]:
regressor_no_intercept = LinearRegression(fit_intercept=False, n_jobs=1)

def predict_affiliations(text, num=0):
    bigrams = get_bigrams(text)
    counts = [bigrams.count(top_bigram) for top_bigram in top_bigrams]
    df = pd.DataFrame(top_bigrams, columns=['bigrams'])
    df['counts'] = counts
    res_df_1000['text{}'.format(num)] = counts
    df_counts_only = df.drop('bigrams', axis=1)
    df_counts_only = ((df_counts_only.T - std_params['mean']) / std_params['std']).T
    political_Ngrams_1000_std['text{}'.format(num)] = df_counts_only['counts']
    df = pd.DataFrame(data=np.column_stack((top_bigrams, df_counts_only)), columns=['bigrams', 'counts'])
#     df = df.fillna(0)
    X = betas.reshape(-1, 1)
    y = np.array((np.array(df['counts']) - alphas)).ravel()
    regressor_no_intercept.fit(X, y)
    
    return regressor_no_intercept.coef_[0]

In [479]:
test_df = pd.read_csv('test_get_the_facts.csv')
test_df

,text
0,Over 3 million UK jobs are linked to our trade...
1,"In the EU you can find work, holiday and retir..."
2,If we leave the EU experts predict that the ec...
3,The Trades Union Congress says the average wag...
4,Being in the EU means you pay less for your we...
5,It's estimated that saves your family £350 a y...
6,The Governor of the Bank of England says Brexi...
7,9 out of 10 economic experts say our economy w...
8,Overall we get more out than we pay in. Britai...
9,The UK economy benefits from investment worth ...


In [480]:
text = ' '.join(test_df['text'].values)

In [481]:
predict_affiliations(text)

-30.818809838983945

In [482]:
test_df = pd.read_csv('test_key_speeches_combined.csv')
test_df

,text
0,We all agree that it is vital that everyone is...
1,"If we Vote Leave on 23 June, the Government wi..."
2,Thirteen Government ministers and senior Conse...
3,Michael Gove and Dominic Raab have today made ...
4,"Michael Gove, Boris Johnson, Gisela Stuart and..."
5,"Michael Gove, Boris Johnson, and Gisela Stuart..."
6,A STRONGER NHS AND MORE MONEY FOR THOSE IN NEE...
7,Liam Fox MP delivered a speech today at Vote L...
8,"Statement by Michael Gove, Boris Johnson, Prit..."
9,"On 23 June, Sun readers can decide whether we ..."


In [483]:
rows = []

n = len(test_df)
for index in tqdm(range(n)):
    row = test_df.iloc[index]
    transcript = row['text']
#     speaker = row['name']
    rows.append([transcript, predict_affiliations(transcript)])
    
aff_df = pd.DataFrame(rows, columns=['transcript', 'affiliations'])
aff_df

100%|██████████| 24/24 [00:11<00:00,  2.15it/s]


,transcript,affiliations
0,We all agree that it is vital that everyone is...,-7.093162
1,"If we Vote Leave on 23 June, the Government wi...",-245.395064
2,Thirteen Government ministers and senior Conse...,-74.672320
3,Michael Gove and Dominic Raab have today made ...,-181.184005
4,"Michael Gove, Boris Johnson, Gisela Stuart and...",-164.139947
5,"Michael Gove, Boris Johnson, and Gisela Stuart...",-56.144722
6,A STRONGER NHS AND MORE MONEY FOR THOSE IN NEE...,-63.082439
7,Liam Fox MP delivered a speech today at Vote L...,-61.221553
8,"Statement by Michael Gove, Boris Johnson, Prit...",-171.397313
9,"On 23 June, Sun readers can decide whether we ...",-32.556011


In [484]:
aff_df.to_csv('key_speeches_prediction.csv', index=False)

In [485]:
political_Ngrams_1000_std.to_csv('final_dataframe_party_std_after.csv', index=False)

In [486]:
res_df_1000.to_csv('raw_bigrams_counts.csv', index=False)